In [7]:
import regex as re
import pandas as pd
import os
import random
import numpy as np
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='your module name')

In [12]:
df = pd.read_csv('data.csv',index_col=0)
index = df.index
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import casual_tokenize
np.random.seed(42)

counter = CountVectorizer(tokenizer=casual_tokenize)
bow_docs = pd.DataFrame(counter.fit_transform(raw_documents=df.seg_text)
                        .toarray(), index=index)
column_nums, terms = zip(*sorted(zip(counter.vocabulary_.values(),
                                     counter.vocabulary_.keys())))
bow_docs.columns = terms
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import casual_tokenize
tfidf = TfidfVectorizer(tokenizer=casual_tokenize,)
tfidf_docs =tfidf.fit_transform(raw_documents=df.text).toarray()
tfidf_docs = tfidf_docs - tfidf_docs.mean(axis=0)


try:
    print('rumor0:'+df.loc['rumor0'].text)
    print(bow_docs.loc['rumor0'][bow_docs.loc['rumor0'] > 0].head())
except KeyError:
    print('rumor0!:'+df.loc['rumor0!'].text)
    print(bow_docs.loc['rumor0!'][bow_docs.loc['rumor0!'] > 0].head())

/Users/stevenyu/opt/anaconda3/envs/d2l/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/stevenyu/opt/anaconda3/envs/d2l/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


rumor0!:脑出血不能引起病人剧烈的头痛。
不能    1
剧烈    1
头痛    1
引起    1
病人    1
Name: rumor0!, dtype: int64


In [14]:
sentiments = df.sentiments
sentiment_dict = np.load('sentiment_dict.npy',allow_pickle=True).item()

In [18]:
from sklearn.decomposition import LatentDirichletAllocation as LDiA
ldia100 = LDiA(n_components=100, learning_method='batch')
ldia100 = ldia100.fit(bow_docs)
ldia100_topic_vectors = ldia100.transform(bow_docs)
columns100 = ['topic{}'.format(i) for i in range(ldia100.components_.shape[0])]
ldia100_topic_vectors_doc = pd.DataFrame(ldia100_topic_vectors,index=index, columns=columns100)
ldia100_topic_vectors_sentiment = np.array([ldia100_topic_vectors[i]*sentiments[i] for i in range(len(sentiments))])
ldia100_topic_vectors_sentiment_doc = pd.DataFrame(ldia100_topic_vectors_sentiment,index=index, columns=columns100)

ldia300 = LDiA(n_components=300, learning_method='batch')
ldia300 = ldia100.fit(bow_docs)
ldia300_topic_vectors = ldia300.transform(bow_docs)
columns300 = ['topic{}'.format(i) for i in range(ldia300.components_.shape[0])]
ldia300_topic_vectors_doc = pd.DataFrame(ldia300_topic_vectors,index=index, columns=columns300)
ldia300_topic_vectors_sentiment = np.array([ldia300_topic_vectors[i]*sentiments[i] for i in range(len(sentiments))])
ldia300_topic_vectors_sentiment_doc = pd.DataFrame(ldia300_topic_vectors_sentiment,index=index, columns=columns300)

In [19]:
from sklearn.decomposition import PCA
k=min(len(tfidf.vocabulary_),len(df))
k=100
pca = PCA(n_components=k)
pca_docs = pca.fit_transform(tfidf_docs)
base = sum(pca.singular_values_)
nums = pca.singular_values_.tolist()
pca = PCA(n_components=k)
pca_docs = pca.fit_transform(tfidf_docs)
pca_topic_vectors_100 = pca.transform(tfidf_docs)
columns = ['topic{}'.format(i) for i in range(pca_docs.shape[1])]
pca_topic_vectors_100_doc = pd.DataFrame(pca_topic_vectors_100, columns=columns, index=index)
pca_topic_vectors_100_sentiment = np.array([pca_topic_vectors_100[i]*sentiments[i] for i in range(len(sentiments))])
pca_topic_vectors_100_sentiment_doc = pd.DataFrame(pca_topic_vectors_100_sentiment, columns=columns, index=index)
k=300
pca = PCA(n_components=k)
pca_docs = pca.fit_transform(tfidf_docs)
base = sum(pca.singular_values_)
nums = pca.singular_values_.tolist()
pca = PCA(n_components=k)
pca_docs = pca.fit_transform(tfidf_docs)
pca_topic_vectors_300 = pca.transform(tfidf_docs)
columns = ['topic{}'.format(i) for i in range(pca_docs.shape[1])]
pca_topic_vectors_300_doc = pd.DataFrame(pca_topic_vectors_300, columns=columns, index=index)
pca_topic_vectors_300_sentiment = np.array([pca_topic_vectors_300[i]*sentiments[i] for i in range(len(sentiments))])
pca_topic_vectors_300_sentiment_doc = pd.DataFrame(pca_topic_vectors_300_sentiment, columns=columns, index=index)

In [ ]:
tfidf.vocabulary_
#根据词项的频率对词汇表进行排序
#当对某个不按照最左边元素排序的序列解压并在排序后重新压缩时，可以使用zip(*sorted(zip(...)))
column_nums , terms = zip(*sorted(zip(tfidf.vocabulary_.values(), tfidf.vocabulary_.keys())))
weights = pd.DataFrame(pca.components_, columns=terms,
                       index = ['topic{}'.format(i) for i in range(pca.components_.shape[0])])
pd.options.display.max_columns = 8

In [20]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=100,n_iter=100)
svd_topic_vectors_100 = svd.fit_transform(tfidf_docs)
svd_topic_vectors_100 = (svd_topic_vectors_100.T / np.linalg.norm(svd_topic_vectors_100,axis=1)).T
svd_topic_vectors_100_doc = pd.DataFrame(svd_topic_vectors_100, columns=columns, index=index)
svd_topic_vectors_100_sentiment = np.array([svd_topic_vectors_100[i]*sentiments[i] for i in range(len(sentiments))])
svd_topic_vectors_100_sentiment_doc = pd.DataFrame(svd_topic_vectors_100_sentiment, columns=columns, index=index)

svd = TruncatedSVD(n_components=300,n_iter=100)
svd_topic_vectors_300 = svd.fit_transform(tfidf_docs)
svd_topic_vectors_300 = (svd_topic_vectors_300.T / np.linalg.norm(svd_topic_vectors_300,axis=1)).T
svd_topic_vectors_300_doc = pd.DataFrame(svd_topic_vectors_300, columns=columns, index=index)
svd_topic_vectors_300_sentiment = np.array([svd_topic_vectors_300[i]*sentiments[i] for i in range(len(sentiments))])
svd_topic_vectors_300_sentiment_doc = pd.DataFrame(svd_topic_vectors_300_sentiment, columns=columns, index=index)

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split

def lda_predic (name,vector,df,over_sampling=False):
    X_train,X_test,y_train,y_test = train_test_split(vector,df.rumor,test_size=0.2 ,random_state=42)
    if over_sampling:
        smote = SMOTE(random_state=42)
        # 进行过采样
        X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
        print("过采样前训练集中类别0的数量：", sum(y_train==0))
        print("过采样前训练集中类别1的数量：", sum(y_train==1))

        print("过采样后训练集中类别0的数量：", sum(y_train_res==0))
        print("过采样后训练集中类别1的数量：", sum(y_train_res==1))

    lda = LDA(n_components=1)
    lda = lda.fit(X_train_res, y_train_res)
    df['name'] = lda.predict(vector)
    round(float(lda.score(X_test, y_test)), 3)
    from sklearn import metrics
    y_true = df.loc[y_test.index].rumor
    y_pred = df.loc[y_test.index].LSA16_rumor
    loss_total = 0
    acc = metrics.accuracy_score(y_true, y_pred)
    report = metrics.classification_report(y_true, y_pred, target_names=["non-rumor","rumor"], digits=4)
    confusion = metrics.confusion_matrix(y_true, y_pred)
    print(report)
    with open("../Chinese-Text-Classification-Pytorch/report/"+name+'.txt', 'w', encoding='utf-8') as f:
        print(report,file=f)
    print(confusion)
    with open("../Chinese-Text-Classification-Pytorch/report_matrix/"+name+'_matrix'+'.txt', 'w', encoding='utf-8') as f:
        print(confusion,file=f)
    np.savez("../Chinese-Text-Classification-Pytorch/report_matrix/"+name+'_matrix'+'.npz', matrix= confusion)    